Luego de haber realizado las peticiones con el HTML Scrapper, y obtenido cada par clave valor de la tabla de caracteristicas con el HTML Spec Table Parser, vamos a descargar los archivos resultantes de este ultimo, y procesarlos en esta notebook.

Para esto, nos autenticamos con Google Cloud Storage, y descargamos dichos archivos.

In [29]:
project_id = 'cryptic-opus-335323'
bucket_name = 'bdm-unlu'
from google.colab import auth
auth.authenticate_user()
!rm -r html_parser
!gsutil cp -r gs://{bucket_name}/html_parser/ .

Copying gs://bdm-unlu/html_parser/parsed_result_0.pkl...
Copying gs://bdm-unlu/html_parser/parsed_result_1.pkl...
Copying gs://bdm-unlu/html_parser/parsed_result_10.pkl...
Copying gs://bdm-unlu/html_parser/parsed_result_11.pkl...
/ [4 files][805.7 KiB/805.7 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://bdm-unlu/html_parser/parsed_result_2.pkl...
Copying gs://bdm-unlu/html_parser/parsed_result_3.pkl...
Copying gs://bdm-unlu/html_parser/parsed_result_4.pkl...
Copying gs://bdm-unlu/html_parser/parsed_result_5.pkl...
Copying gs://bdm-unlu/html_parser/parsed_result_6.pkl...
Copying gs://bdm-unlu/html_parser/parsed_result_7.pkl...
Copying gs://bdm-unlu/html_parser/parsed_result_8.pkl...
Copying gs://bdm-unlu/html_parser/pars

Utilizando el comando cat de Linux, respetando la numeración de los archivos, los concatenamos, resultando en el archivo html_parser/results.pkl.gz

In [30]:
!cat html_parser/parsed_result_0.pkl html_parser/parsed_result_1.pkl html_parser/parsed_result_2.pkl html_parser/parsed_result_3.pkl html_parser/parsed_result_4.pkl html_parser/parsed_result_5.pkl html_parser/parsed_result_6.pkl html_parser/parsed_result_7.pkl html_parser/parsed_result_8.pkl html_parser/parsed_result_9.pkl html_parser/parsed_result_10.pkl html_parser/parsed_result_11.pkl > html_parser/results.pkl.gz

Descomprimimos dicho archivo

In [31]:
!gzip -d html_parser/results.pkl.gz

Cargamos todos los resultados de este archivo en la variable "results". Vemos que en total tenemos 12 resultados, correspondiendo con la cantidad de instancias de Google.

In [32]:
import pickle

open_file_read = open("html_parser/results.pkl", "rb")

raw_results_count = 0
has_load = True
results = []

while has_load:     
  try:
    raw_result = pickle.load(open_file_read)
    raw_results_count += 1
    results.extend(raw_result)
  except Exception as e:
    has_load = False

print("Read {} results.".format(raw_results_count))

    
    

Read 12 results.


Vemos la longitud de esta lista

In [33]:
len(results)

82165

Para verificar, podríamos preguntarnos, cuantos permalinks faltan consultar?

Descargamos el dataset para obtener la cantidad original de permalinks que deberíamos haber solicitado


In [34]:
!gsutil cp gs://{bucket_name}/dataset_trimmer/trimmed_dataset_cf.csv .

import pandas as pd
df = pd.read_csv("trimmed_dataset_cf.csv")

Copying gs://bdm-unlu/dataset_trimmer/trimmed_dataset_cf.csv...
/ [1 files][ 28.6 MiB/ 28.6 MiB]                                                
Operation completed over 1 objects/28.6 MiB.                                     


Si restamos las longitudes, faltan consultar:

In [35]:
len(results) - len(df["permalink"])

0

Con esta función obtenemos la lista univoca de claves que obtuvimos de la tabla de caracteristicas de los inmuebles.

In [36]:
def get_unique_key_list(results):
  key_list = []
  for result in results:
    for key in result[1].keys():
      if key not in key_list:
        key_list.append(key)
  return key_list

key_list = get_unique_key_list(results)
key_list

['Superficie total',
 'Superficie cubierta',
 'Ambientes',
 'Dormitorios',
 'Baños',
 'Cocheras',
 'Cantidad de pisos',
 'Disposición',
 'Antigüedad',
 'Expensas',
 'Departamentos por piso',
 'Número de piso de la unidad',
 'Orientación',
 'Tipo de departamento',
 'Bodegas',
 'Tipo de casa',
 'Superficie de terreno']

Comprobamos si el orden de los resultados, es el mismo que el orden del dataset, utilizando los permalinks. Y vemos que efectivamente es así.

In [37]:
index = 0
for result in results:
  if df.iloc[index]["permalink"] != result[0]:
    print("Diferent order between dataframe and results")
    break
  index += 1

Creamos un diccionario, donde las claves, son cada una de las caracteristicas univocas obtenidas anteriormente

In [38]:
new_data = {}
for key in key_list:
  new_data[key] = []

Y por cada resultado, agregamos en cada clave del diccionario, el valor que contenga para esa clave determinada. Sin importar si este tiene un valor especifico, o un None.

In [39]:
for result in results:
  for key in key_list:
    new_data[key].append(result[1].get(key))

Luego, cada clave del diccionario, la asignamos al dataset, creando una nueva columna.

In [40]:
for key in key_list:
  df[key] = new_data[key]

Resultando en el siguiente dataset.

In [41]:
pd.set_option('display.max_colwidth', 150)
df.head()

,Unnamed: 0,Unnamed: 0.1,title,permalink,city,state,latitude,longitude,neighborhood,seller_id,seller_city,seller_state,real_estate_agency,seller_cancelations,seller_claims,seller_handling_time,seller_sales,currency_id,price,item_condition,has_air_conditioning,has_telephone_line,bedrooms,covered_area,full_bathrooms,rooms,total_area,operation,property_type,with_virtual_tour,Superficie total,Superficie cubierta,Ambientes,Dormitorios,Baños,Cocheras,Cantidad de pisos,Disposición,Antigüedad,Expensas,Departamentos por piso,Número de piso de la unidad,Orientación,Tipo de departamento,Bodegas,Tipo de casa,Superficie de terreno
0,0,0,Departamento Venta 3 Ambientes Palermo Amenities Cochera,https://departamento.mercadolibre.com.ar/MLA-1116767893-departamento-venta-3-ambientes-palermo-amenities-cochera-_JM,Capital Federal,Capital Federal,-34.579563,-58.433160,Palermo Hollywood,314431007,Palermo,Capital Federal,True,0.0,0.0,0.0,0.0,USD,270000.0,Usado,No,No,2.0,85 m²,3.0,3.0,85 m²,Venta,Departamento,No,85 m²,85 m²,3,2,3,1,7,Frente,10 años,20000 ARS,None,None,None,None,None,None,None
1,1,1,Venta Departamento 3 Ambientes Piso 15 Palermo,https://departamento.mercadolibre.com.ar/MLA-1107347883-venta-departamento-3-ambientes-piso-15-palermo-_JM,Capital Federal,Capital Federal,-34.573490,-58.436577,Palermo Hollywood,466036843,Monserrat,Capital Federal,True,0.0,0.0,0.0,0.0,USD,148500.0,Usado,NaN,NaN,2.0,46 m²,1.0,3.0,55 m²,Venta,Departamento,No,55 m²,46 m²,3,2,1,None,None,None,50 años,12.5 ARS,None,None,None,None,None,None,None
2,2,2,Venta Departamento 3amb Con Cochera Triplex Palermohollywood,https://departamento.mercadolibre.com.ar/MLA-1113461025-venta-departamento-3amb-con-cochera-triplex-palermohollywood-_JM,Capital Federal,Capital Federal,-34.585620,-58.440120,Palermo Hollywood,419112633,Palermo Chico,Capital Federal,True,0.0,0.0,0.0,0.0,USD,215000.0,Usado,No,Sí,2.0,93 m²,3.0,3.0,102 m²,Venta,Departamento,NaN,102 m²,93 m²,3,2,3,1,3,Frente,5 años,16000 ARS,3,4,Sur,None,None,None,None
3,3,3,Departamento - Palermo Hollywood - Venta Monoambiente Quartier Dorrego,https://departamento.mercadolibre.com.ar/MLA-921699674-departamento-palermo-hollywood-venta-monoambiente-quartier-dorrego-_JM,Capital Federal,Capital Federal,-34.577396,-58.439194,Palermo Hollywood,694286242,Belgrano,Capital Federal,True,0.0,0.0,0.0,0.0,USD,180000.0,Nuevo,Sí,No,0.0,40 m²,1.0,1.0,40 m²,Venta,Departamento,No,40 m²,40 m²,1,None,1,None,None,None,0 años,9850 ARS,None,None,None,None,None,None,None
4,4,4,Departamento - Palermo Hollywood,https://departamento.mercadolibre.com.ar/MLA-1116714973-departamento-palermo-hollywood-_JM,Capital Federal,Capital Federal,-34.579685,-58.429800,Palermo Hollywood,418539414,Palermo Chico,Capital Federal,True,0.0,0.0,0.0,0.0,USD,135000.0,Nuevo,No,No,1.0,46 m²,1.0,2.0,52 m²,Venta,Departamento,No,52 m²,46 m²,2,1,1,None,None,None,0 años,10000 ARS,None,None,None,None,None,None,None


Lo guardamos a csv, y lo almacenamos en el bucket.

In [42]:
df.to_csv("dataset.csv")
!gsutil cp dataset.csv gs://{bucket_name}/html_spec_table_processor/dataset.csv

Copying file://dataset.csv [Content-Type=text/csv]...
/ [1 files][ 33.2 MiB/ 33.2 MiB]                                                
Operation completed over 1 objects/33.2 MiB.                                     
